# Who is heading for Diabetes?

This is the predictive part of the 2017 Melbourne Datathon.

The task is to predict the probability that a patient will be dispensed a drug related to Diabetes post 2015. This is quite important research as it will be an early warning system for doctors so intervention can potentially be made before it is too late.

Use the patients that we have provided all the records for to build your model, then see how it performs on these unseen people.

For patient ID'S 279,201 to 558,352 you need to submit a file with 2 columns, the Patient_ID and the probability in the range [0-1]. The file will have 279,153 rows including the header row. An example submission file is provided for download.

In [1]:
import tqdm

import pandas as pd
import numpy as np
import pickle
import sqlite3

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.preprocessing import normalize, StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

## Data retrieval functions

In [2]:
conn = sqlite3.connect("../data/datasets.db")

In [3]:
def retrieve_patient_data(connection, patient_id):
    """
    Return the patient data.
    """
    SQL = """
SELECT *
FROM transactions a
LEFT OUTER JOIN ChronicIllness_LookUp b 
    ON a.Drug_ID = b.MasterProductID 
LEFT OUTER JOIN patients c
    ON a.Patient_ID = c.Patient_ID
LEFT OUTER JOIN classification d
    ON a.Patient_ID = d.Patient_ID
LEFT OUTER JOIN social e
    ON c.postcode = e.postcode
WHERE a.Patient_ID = {}
AND a.prescription_week < '2016-01-01'
ORDER BY prescription_week
    """.format(patient_id)

    return pd.read_sql_query(SQL, connection)

## Feature extraction functions

In [4]:
gender_map = {'F': 1, 'M': 0, 'U': 0.5}

def gender(patient_data):
    return gender_map[patient_data.gender[0]]

def age(patient_data):
    patient_age = 2016 - patient_data.year_of_birth[0]
    if patient_age > 100: 
        return 0.5 
    else: 
        return patient_age / 100.

def socio_score(patient_data):
    score = patient_data.disadvantage_score[0]
    if isinstance(score, str):
        return 1.
    if score is None:
        return 1.
    return float(score) / 1000.

def diabetes(patient_data):
    return float(patient_data.ChronicIllness.str.contains('Diabetes').any())

def lipids(patient_data):
    return float(patient_data.ChronicIllness.str.contains('Lipids').any())

def hypertension(patient_data):
    return float(patient_data.ChronicIllness.str.contains('Hypertension').any())

def heart_failure(patient_data):
    return float(patient_data.ChronicIllness.str.contains('Heart Failure').any())

def osteoporosis(patient_data):
    return float(patient_data.ChronicIllness.str.contains('Osteoporosis').any())

def depression(patient_data):
    return float(patient_data.ChronicIllness.str.contains('Depression').any())

## Compute some basic features of the data 

In [5]:
def feature_extract(patient_frame):
    """
    Form a feature dictionary
    """
    
    return {'gender': gender(patient_frame), 
            'age': age(patient_frame), 
            'socio_score': socio_score(patient_frame),
            'diabetes': diabetes(patient_frame),
            'lipids': lipids(patient_frame),
            'hypertension': hypertension(patient_frame),
            'heart_failure':heart_failure(patient_frame),
            'osteoporosis': osteoporosis(patient_frame),
            'depression': depression(patient_frame),
            'target': patient_frame.Target[0]}
    

## Extract features of the data

Perform a random sample of patients.

In [ ]:
n = 10000
patient_ids = np.random.randint(0, 279201, n)
patient_data = []
for patient_id in tqdm.tqdm_notebook(patient_ids):
    patient_data.append(retrieve_patient_data(conn, patient_id))

Extract features into a feature dataframe.

In [ ]:
features = []
for patient in tqdm.tqdm_notebook(patient_data): 
    features.append(feature_extract(patient))
feature_frame = pd.DataFrame(features)

Store the features and ?patient data? to disk.

In [ ]:
feature_frame.to_csv('../submissions/features.csv')

with open('../submissions/patient_data.pkl', 'wb') as fh:
    pickle.dump(patient_data, fh)

In [6]:
feature_frame = pd.read_csv('../submissions/features.csv')

with open('../submissions/patient_data.pkl', 'rb') as fh:
    patient_data = pickle.load(fh)

## Adhoc features

An example of updating the feature frame.

In [7]:
def custom_feature(patient_frame):
    
    codes = {
     'A': 0, # Unknown
     'C': 0, # Unknown   
     'Z': 0, # Unknown
     'N': 0, # NHS Script
     'P': 0, # Private Script
     'B': 0, # Doctors Bag Script
     'T': 0, # Schedule Three Recordable Script
     'R': 0, # Repatriation Script
     'D': 0, # Dental Script
     'E': 0, # Optometrist Script
     'U': 0, # Nurse Practitioner Script
     'F': 0  # Midwife Script
    }
    
    codes.update(patient_frame.SourceSystem_Code.value_counts(normalize=True).to_dict())
    return codes

features = []
for patient in tqdm.tqdm_notebook(patient_data): 
    features.append(custom_feature(patient))

features = pd.DataFrame(features)
features.columns = ['system_code_'+ x.lower() for x in features.columns]
feature_frame = pd.concat( [feature_frame, features], axis=1)

In [40]:
def mean_script_time(patient_frame):
    script_week_diff = pd.to_datetime(patient_frame.Prescription_Week).diff()
    return script_week_diff[script_week_diff > pd.Timedelta(0)].mean() / pd.Timedelta(days=365*6)

features = []
for patient in tqdm.tqdm_notebook(patient_data): 
    features.append(mean_script_time(patient))
feature_frame['mean_script_time'] = features

## Explore the feature we have extracted

Note: not sure here - trying out something from scikit learn but it may not be sensible.

In [41]:
means = feature_frame.groupby('target').mean()
means

,Unnamed: 0,age,depression,diabetes,gender,heart_failure,hypertension,lipids,osteoporosis,socio_score,...,system_code_d,system_code_e,system_code_f,system_code_n,system_code_p,system_code_r,system_code_t,system_code_u,system_code_z,mean_script_time
target,,,,,,,,,,,,,,,,,,,,,
0,5001.409290,0.510346,0.343941,0.060595,0.576201,0.309071,0.438917,0.578639,0.071934,0.989911,...,0.0,0.0,0.919429,0.0,0.0,0.0,0.0,0.0,0.002718,0.056454
1,4990.790323,0.509538,0.290323,0.938821,0.511123,0.370968,0.543938,0.855395,0.037820,0.981688,...,0.0,0.0,0.898247,0.0,0.0,0.0,0.0,0.0,0.000545,0.044154


## Try out a set of different classifiers

Note: the feature matrix is usually transformed to have zero mean and unit standard deviation.

In [43]:
X = feature_frame.drop('target', axis=1).values
y = feature_frame.target.values

X_transformed = StandardScaler().fit_transform(X)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

Partition the data into test and train datasets.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.5, random_state=0)

In [ ]:
classifiers = [
    ('Random Forrest', RandomForestClassifier()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Adaboost',AdaBoostClassifier() ),
    ('SVM',SVC(probability=True))]

for name, clf in classifiers:
    print('Classifier: {}'.format(name))
    model = clf.fit(X_train, y_train)
    y_true, y_pred = y_test, clf.predict(X_test) 
    print(classification_report(y_true, y_pred))


## Form a submission

Perform the prediction in 1000 patient "chunks" to speed up the processing.

In [ ]:
submission = pd.read_csv('../../submissions/diabetes_submission_example.csv')

In [ ]:
chunks = submission.groupby(np.arange(len(submission)) // 1000)

In [ ]:
for group, frame in tqdm.tqdm_notebook(chunks):
    
    # Extract the features
    data = [feature_extract(patient_data(conn, x)) for x in frame.Patient_ID.values]
    
    # Construct prediction X matrix
    pred_x = np.vstack([x[0] for x in data])
    
    # Make sure we don't have nans in the data
    pred_x[np.isnan(pred_x)] = 0
    
    # Apply the standard transform prior to fitting. 
    pred_x = StandardScaler().fit_transform(pred_x)
    
    # Fit the model
    submission.Diabetes[frame.index] = model.predict_proba(pred_x)[:, 0]

In [ ]:
submission.to_csv('../../submissions/kaggle.csv', index=False)